### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

### MechInterp

In [ ]:
step 1: two prompts
step 2: record the interdimate activations of the two prompts
step 3: patch

In [ ]:
softmax(x@W_E@W_QK)@x@W_E^T@W_OV

In [ ]:
node > pod > container

### Engineering

In [ ]:
api servier, ectd, scheduler, control manager

In [ ]:
def compute_total_memory(model):
    total_memory = 0
    for param in model.parameters():
        total_memory += param.storage.size() + param.numel()
    
    return total_memory

In [2]:
import socketserver

In [3]:
class EchoRequestHandler(socketserver.StreamRequestHandler):
    def handle(self):
        pass

In [ ]:
with socketserver.ThreadingTCPServer(
    (MASTER_HOST, MASTER_PORT),
    EchoRequestHandler
) as server:
    server.serve_forever()

elastic driver, notification service, notification manager, notification client, state, hostdiscovery

In [4]:
import copy

In [5]:
class RegularState:
    def __init__(self, states):
        self.states = states
        self._saved_states = states
    
    def commit(self):
        new_states = {}
        for name in self._saved_states.key():
            new_states[name] = getattr(self, name)
        self._saved_states = new_states
    
    def restore(self):
        for name, value in self._saved_states:
            setattr(self, name, value)

In [7]:
class State(RegularState):
    def __init__(self, model, optim, **kwargs):
        kwargs.update({"model": model, "optim": optim})
        handlers, regular_states = get_handlers(kwargs)
        
        for name, handler in handlers:
            setattr(self, name, handler)
        
        self._handlers = handlers
        super().__init__(self, states=regular_states)
    
    def commit(self):
        for handler in self._handlers.values():
            handler.commit()
        super().commit(self)
    
    def restore(self):
        for handler in self._handlers.values():
            handler.restore()
        super().restore(self)

In [11]:
class ModelStateHandler:
    def __init__(self, model):
        self.set_value(model)
        self._model_state_dict = copy.deepcopy(
            model.state_dict()
        )
    
    def set_value(self, value):
        self.value = value
    
    def commit(self):
        self._model_state_dict = copy.deepcopy(
            self.value.state_dict()
        )
        
    def restore(self):
        self.value.load_state_dict(self._model_state_dict)

In [10]:
def get_handler(v):
    for handler_type, handler_cls in handler_registry:
        if isinstance(v, handler_cls):
            return handler_cls(v)
    return None

In [8]:
def get_handlers(states):
    handlers = {}
    remainders = {}
    
    for k, v in states:
        handler = get_handler(v)
        if handler is None:
            remainders[k] = v
        else:
            handlers[k] = handler
        
    return handlers, remainders

In [13]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embedding, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        
        num_embedding_per_partrition = num_embedding // world_size
        self.weight = nn.Parameter(torch.randn(
            num_embedding_per_partrition,
            embedding_dim
        ))
        
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            num_embedding_per_partrition
        )
    
    def get_vocab_range(self, num_embedding_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*num_embedding_per_partrition
        end_idx = start_idx + num_embedding_per_partrition
        return start_idx, end_idx
    
    def forward(self, tokens):
        mask = (tokens < self.vocab_start_idx) | (tokens > self.vocab_end_idx)
        masked_tokens = tokens - self.vocab_start_idx
        masked_tokens[mask] = 0.
        
        embeddings = F.embedding(tokens, weight=self.weight)
        mask_idxs = torch.where(mask == True)[1]
        embeddings[:, mask_idxs, :] = 0.
        
        torch.distributed.all_reduce(
            embeddings
        )
        
        return embeddings

In [ ]:
broadcast, reduce, scatter, gather

In [ ]:
scheduler, api server, control manager, ectd

In [ ]:
def split_model(model, balances, devices):
    layers = {}
    partrition_idx = 0
    partritions = []
    
    for name, layer in model.named_children():
        layers[name] = layer
        
        if len(layers) == balances[partrition_idx]:
            partrition = nn.Sequential(layers)
            partrition.to(devices[partrition_idx])
            partritions.append(partrition)
            layers.clear()
            partrition_idx += 1
    
    return partritions

In [14]:
from queue import Queue

In [15]:
q = Queue()

In [ ]:
for x in xs:
    q.put(x)

In [ ]:
ready, succeed, failed, cooldown, blacklist

### MechInterp

In [ ]:
other=query, no mask



In [ ]:
model.W_O[layer_idx, head_idx]

In [ ]:
W_E = model.W_E
W_U = model.W_U


W_Q = model.W_Q[1, 4]
W_K = model.W_K[1, 4]

W_O = model.W_O[0, 7]
W_V = model.W_V[0, 7]

Q = W_E @ W_Q

K = W_E @ W_V @ W_O @ W_K

In [ ]:
(head_1 + head_2 + head_3) + bias

In [ ]:
model.reset_hooks()

In [ ]:
probs @ W_V @ W_O

In [ ]:
result = the output of each heads
attn_out = the output of an attention layer

In [ ]:
step 1: 2 prompts
step 2: record all the interdimate activations
step 3: iteartive
step 4: logit diff

In [ ]:
induction_stripe = attn_weights.diagonal(-2, -1,)

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [ ]:
seq_len = clean_tokens.shape[-1]

In [ ]:
patched_residual_stream_diff = torch.zeros(n_layers, seq_len)

In [16]:
from functools import partial

In [ ]:
def patch_resid(activations, hook, position_idx, layer_idx):
    activations[:, position_idx, :] = clean_activations[hook.name][]

In [17]:
from transformer_lens.utils import get_act_name

In [ ]:
correct_token = model.to_single_token("John")
incorrect_token = model.to_single_token("Mary")

In [18]:
def compute_logit_difference(logits, correct_token, incorrect_token):
    last_token_logits = logits[:, -1, :]
    return last_token_logits[correct_token, :] - last_token_logits[incorrect_token, :]

In [ ]:
for layer_idx in range(n_layers):
    for position_idx in range(seq_len):
        hook_func = partial(patch_resid, position_idx=position_idx, layer_idx)
        hook_name = get_act_name("resid_pre", layer_idx)
        patched_logits = model.run_with_hooks(
            corrupted_prompt,
            hook_fwd=[(hook_name, hook_func)]
        )
        logit_diff = compute_logit_difference(patched_logits, correct_token, incorrect_token)
        patched_residual_stream_diff[layer_idx][position_idx] = logit_diff

In [ ]:
step 1: record all the interdimate activations
step 2: logit differnece direction
step 3: project residual stream to 

### Engineering

In [20]:
import pytest

In [ ]:
@pytest.mark.reparametrize(
    "input, output",
    [(1, 1), (2, 4)]
)
def test_square(input, output):
    assert square(input) == output

### Mech

In [ ]:
W_E @ QK @ W_OV @ W_E

In [ ]:
step 1: record all the interdimate activations
step 2: attention pattern
step 3: induction head
step 4: decompose attention scores
step 5: identify which pair contribute the most to the attention score
step 6: trace back the previous attention head

In [ ]:
step 1: convert the command to string
step 2: open a ssh session
step 3: run the string

In [ ]:
save
restore
reset
commit
sync

In [21]:
def probability_score(image_embedding, text_embedding):
    image_embedding_norm = image_embedding.norm(dim=-1, keepdim=True)
    image_embedding = image_embedding / image_embedding_norm
    
    text_embedding_norm = text_embedding.norm(dim=-1, keepdim=True)
    text_embedding = text_embedding / text_embedding_norm
    
    similarities = image_embedding @ text_embedding.T
    probs = F.softmax(similarities, dim=-1)
    return probs

In [22]:
class BatchNorm(nn.Module):
    def __init__(self, mom, eps):
        super().__init__()
        self.mom = mom
        self.eps = eps
        self.mults = nn.Parameter(torch.ones(1))
        self.adds = nn.Parameter(torch.zeros(1))
        self.register_buffer("means", torch.zeros(1))
        self.register_buffer("vars", torch.ones(1))
    
    def update_stats(self, x):
        mean = x.mean(dim=0, keepdim=True)
        var = x.var(dim=0, keepdim=True)
        
        self.mean.lerp_(mean, self.mom)
        self.var.lerp_(var, self.mom)
        return mean, var
    
    def forward(self, x):
        with torch.no_grad():
            mean, var = self.update_stats(x)
        
        x = (x - mean) / (self.eps + var)
        x = self.adds + self.mults*x
        return x